In [1]:
import openai

In [2]:
import os

In [5]:
from openai import OpenAI

In [7]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv('OPENROUTER_API_KEY'),
)

In [9]:
response = client.chat.completions.create(
  model="google/gemma-2-9b-it:free",
  messages=[
    {
      "role": "user",
      "content": "is it too late to join the course?",
    },
  ]
)

response.choices[0].message.content

"Please provide me with more context! I need to know which course you're referring to in order to tell you if it's too late to join. \n\nPlease tell me:\n\n* **The name of the course**\n* **The institution or organization offering the course**\n* **The start date of the course**\n* **The current date**\n\n\nOnce I have this information, I can help you determine if it's still possible to enroll."